<a href="https://colab.research.google.com/github/gagan3012/notebooks/blob/master/Zero_Shot_learning_experiment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install newspaper3k==0.2.8 transformers

     |████████████████████████████████| 211 kB 13.1 MB/s 
     |████████████████████████████████| 3.1 MB 39.0 MB/s 
     |████████████████████████████████| 81 kB 6.3 MB/s 
     |████████████████████████████████| 7.4 MB 15.3 MB/s 
     |████████████████████████████████| 87 kB 5.7 MB/s 
     |████████████████████████████████| 895 kB 37.1 MB/s 
     |████████████████████████████████| 61 kB 511 kB/s 
     |████████████████████████████████| 3.3 MB 42.2 MB/s 
     |████████████████████████████████| 596 kB 39.8 MB/s 
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13552 sha256=b56599aa3851c18d1a38b2ea245ff64580500559049aba5bf7cc8ebbd329bf5e
  Stored in directory: /root/.cache/pip/wheels/df/67/41/faca10fa501ca010be41b49d40360c2959e1c4f09bcbfa37fa
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3356 sha256=c12717c9489ca7db6240463a341aa4c5788e8fbd0a093d240586939e6d9dc910
  Stored in directory: /root/.cache/pip/wheels/7f/d4/8f/

####ZSL
This is the checkpoint for bart-large after being trained on the MultiNLI (MNLI) dataset.

Additional information about this model:

The bart-large model page
BART: Denoising Sequence-to-Sequence Pre-training for Natural Language Generation, Translation, and Comprehension
BART fairseq implementation
NLI-based Zero Shot Text Classification
Yin et al. proposed a method for using pre-trained NLI models as a ready-made zero-shot sequence classifiers. The method works by posing the sequence to be classified as the NLI premise and to construct a hypothesis from each candidate label. For example, if we want to evaluate whether a sequence belongs to the class "politics", we could construct a hypothesis of This text is about politics.. The probabilities for entailment and contradiction are then converted to label probabilities.

This method is surprisingly effective in many cases, particularly when used with larger pre-trained models like BART and Roberta. See this blog post for a more expansive introduction to this and other zero shot methods, and see the code snippets below for examples of using this model for zero-shot classification both with Hugging Face's built-in pipeline and with native Transformers/PyTorch code.

In [2]:
import pandas as pd

data = pd.read_csv("/content/new_articles.csv")
df= data[["url","content","title"]]

pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

df.head(1)

,url,content,title
0,https://www.theguardian.com/us-news/2016/jul/13/alton-sterling-son-cameron-protesters-baton-rouge,"The son of a Louisiana man whose father was shot and killed at range by Baton Rouge police asked protesters for “peace” and “no violence, none whatsoever”. Cameron Sterling, the son of Alton Sterling, whose death at the hands of police was caught on video, spoke about his father and protests. Alton Sterling was killed by Baton Rouge police on 5 July at a convenience store, where he was said to be selling CDs. Baton Rouge police said in a statement that police were called to the convenience store because Sterling had allegedly threatened another patron with a gun. The press conference on Wednesday is Cameron’s first since he broke into sobs at a nationally broadcast press conference with his mother, following his father’s death. “I came to talk to everyone about one: the death of my father. And, two: about how I feel about people in general,” said Cameron, whose distinctly young voice was calm and composed in front of the scrum of reporters at the Triple S convenience store. “People, in general, no matter what the race, should come together as one united family. “No more arguments, violence, crimes,” said Cameron. “Yes, you can protest, but I want everyone to protest the right way. Protest in peace — no guns, no drugs, no alcohol, no violence. Everyone needs to protest in the right way, with peace. No violence, none whatsoever. ” Earlier on Wednesday, in an interview with CBS News, Cameron said he didn’t believe all police were “bad” and that all police “shouldn’t be punished for other police’s crimes”. “The police in Dallas, Texas, they didn’t deserve that,” said Cameron. Alton Sterling’s death spurred a Department of Justice investigation, protests across the country, and was one of the subjects of a march in Dallas on Thursday when a gunman attacked and killed five police officers there. His was the first of two police shootings that week, including a shocking video of the shooting of Minnesota man Philandro Castile, the aftermath of which was broadcast live on Facebook by his girlfriend.","Alton Sterling’s son: ’Everyone needs to protest the right way, with peace’"


In [3]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [4]:
def zsl_exp(df,candidate_labels):
  res_list = []
  for i in range(len(df)):
    sequence_to_classify =df["content"][i]
    print(i)
    #candidate_labels = ["Politics","Terrroism","Religion"]

    data = classifier(sequence_to_classify, candidate_labels, multi_label=False)
    labels = data["labels"]
    scores = data['scores']

    res = {labels[i]: scores[i] for i in range(len(labels))}
    res['seq'] = sequence_to_classify
    res['url'] = df['url'][i]
    res['title'] = df['title'][i]
    res['Number'] = i
    res_list.append(res)

  df_new = pd.DataFrame(res_list)
  
  return df_new

In [5]:
zsl_list= zsl_exp(data,["Politics","Terrroism","Religion"])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [19]:
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.offline import iplot


def plot_zsl(data,themes,selected_theme):
  #zsl_list= zsl_exp(data,themes)
  if selected_theme not in themes:
    return None
  df1 = zsl_list[[selected_theme]]
  def estimate_gaussian(dataset):

    mu = np.mean(dataset) # moyenne cf mu
    sigma = np.std(dataset) # écart_type/standard deviation
    limit = sigma * 1.5

    min_threshold = mu - limit
    max_threshold = mu + limit

    return mu, sigma, min_threshold, max_threshold

  mu, sigma, min_threshold, max_threshold = estimate_gaussian(df1.values)

  condition1 = (df1 < min_threshold)
  condition2 = (df1 > max_threshold)

  outliers1 = np.extract(condition1, df1)
  outliers2 = np.extract(condition2, df1)

  outliers = np.concatenate((outliers1, outliers2), axis=0)

  df_red = df1[df1.values==outliers]
  zsl_list1 = zsl_list[zsl_list[selected_theme].isin(outliers)]
  fig = go.Figure()
  print(zsl_list1[[selected_theme,"title","url"]].to_string())

  fig.add_trace(go.Scatter(x = df1.index, y = df1[selected_theme],
                        mode="markers",marker = dict(color='Blue',
            size=20)
                        ))
  
  fig.add_trace(go.Scatter(x = df_red.index, y = df_red[selected_theme],
                        mode="markers",marker = dict(color='Red',
            size=20)
                        ))
  fig.show()

In [21]:
selected_theme = "Religion" #@param ["Politics","Terrroism","Religion"]

plot_zsl(data=data,themes=["Politics","Terrroism","Religion"],selected_theme=selected_theme)

    Religion                                                                      title                                                                                                                      url
7   0.293407                 Jaffas and daredevils: life on the world’s steepest street                   https://www.theguardian.com/world/2016/jul/23/jaffas-and-daredevils-life-on-the-worlds-steepest-street
10  0.273750          Serbian Olympic rowers sink in ferocious conditions on Rio waters                                    https://www.theguardian.com/sport/2016/aug/06/serbia-rowers-sunk-capsize-rio-olympics
19  0.046677   Golden State Warriors beat Oklahoma City Thunder to return to NBA finals  https://www.theguardian.com/sport/2016/may/31/golden-state-warriors-oklahoma-city-thunder-western-conference-finals-nba
21  0.325230  Pokémon Go: five tricks for pro players that are almost as good as cheats          https://www.theguardian.com/technology/2016/jul/27/pokemon-go-trick